# Statistical Language Modeling with NLTK

### Recommended Reading

- Dan Jurafsky and James H. Martin's __Speech and Language Processing__ ([3rd ed. draft](https://web.stanford.edu/~jurafsky/slp3/))

### Covered Material
- SLP
    - [Chapter 3: N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf) 

<!-- ### Requirements

- [NLTK](https://www.nltk.org/) -->

## 1. Ngrams and Ngram Counting

[n-gram](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of $N$ items from a given sequence of text or speech. An N-gram model can model sequences, notably natural languages, by using the statistical properties of N-grams (based on N-gram counts). 



__Example__:

- character N-grams: mice
- word N-grams: the answer is 42


|                     | 1-gram  | 2-gram  | 3-gram  |
|---------------------|---------|---------|---------|
|                     | unigram | bigram  | trigram |
| *Markov Order*      | 0       | 1       | 2       |
| *Character N-grams* | `['m', 'i', 'c', 'e']` | `['mi', 'ic', 'ce']` | `['mic', 'ice']` | 
| *Word N-grams*      | `['the', 'answer', 'is' , '42']` | `['the answer', 'answer is', ...]` | `['the answer is', ...]` |

You can imagine this as a sliding window with a width of $N$ and a stride to the right of 1  over tokens or characters.

### 1.1. Counting Ngrams

*Frequency List* of a corpus is essentially a unigram count, i.e. the length of the sequences to count is 1. Indeed, N-gram count is just a generalization of *Frequency List* in which the length of the sequences to count is given by $N$ instead of only 1 as default. 

Thus, with N-gram count we can compute the count by taking sequences of 2 items (bigrams), 3 items (trigrams), etc.

#### 1.1.1. Preparing Data
The required data format for n-gram counting is a _list-of-lists_ (lists of sentences consisting of lists of words): 

```
[
     ['the', 'answer', 'is', '42'], 
     ['the', 'mice', 'said']
]
```

#### 1.1.2. Sentence beginning & end tags

>Including sentence boundary markers leads to a better model. To do that we need to augment each sentence with a special symbols for beginning and end of sentence tags (`<s>` and `</s>`, respectively). The beginning of the sentence (BOS) tag gives the bigram context of the first word; and encodes probability of a word to start a sentence. Adding the end of the sentence (EOS) tag delimits the end of a sentence, i.e. it represents the final state of the sequence. The EOS is also used to signaling when the generation of a new sequence can be halted.

> Moreover, for n-grams larger than 2, we’ll need to assume extra context for the contexts to the left and right of the sentence boundaries. For example, to compute trigram probabilities at the very beginning of the sentence, we can use two pseudo-words for the first trigram (i.e. `['<s>', '<s>', w1]`). Alternatively, we can use [back-off](https://en.wikipedia.org/wiki/Katz%27s_back-off_model), and use the `['<s>', w1]` bigram probability. 

**Example**:
`['<s>', 'the', 'answer', 'is', '42', '</s>']`

#### 1.1.3. NLTK Utility Functions
NLTK provides utility functions for padding sequences.

In [ ]:
sent = ['the', 'answer', 'is', '42']

In [ ]:
from nltk.util import pad_sequence

list(pad_sequence(
        sent,  # input sequence
        pad_left=True,
        left_pad_symbol="<s>",
        pad_right=True,
        right_pad_symbol="</s>",
        n=2  # padding for bigrams
    )
)

Another NLTK function wraps this utility function with default arguments to provide a more convenient interface. 

In [ ]:
from nltk.lm.preprocessing import pad_both_ends

list(pad_both_ends(sent, n=2))

#### 1.1.4. Extracting N-grams
NLTK provides a function to extract N-grams from a sequence, which also performs padding, if required arguments are provided.

In [ ]:
from nltk.util import ngrams

list(ngrams(sent, 2))

In [ ]:
bigrams = ngrams(
    sent,
    2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)
list(bigrams)

Additionally, NLTK provides wrapper functions to extract bigrams and trigrams.

In [ ]:
from nltk.util import bigrams, trigrams

print(list(bigrams(sent)))
print(list(trigrams(sent)))

print(list(bigrams(pad_both_ends(sent, n=2))))
print(list(bigrams(sent, 
                   pad_left=True, 
                   left_pad_symbol="<s>", 
                   pad_right=True, 
                   right_pad_symbol="</s>")))

#### 1.1.5. Extracting "Everygrams"
To make an N-gram model more robust, it is usually also trained on lower-order N-grams (i.e. unigrams in case of bigrams). 
NLTK also provides an utility function to extract these together.

Note the `max_len` argument that defines the maximum length of an N-gram.

In [ ]:
from nltk.util import everygrams

print(list(everygrams(sent, max_len=3)))
print(list(everygrams(pad_both_ends(sent, n=2), max_len=2)))

In [ ]:
all_ngrams = everygrams(
    sent,
    min_len=1,
    max_len=2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>"
)
list(all_ngrams)

#### 1.1.6. "Flattening" the Data
For language model training and evaluation (as well as vocabulary extraction) NLTK expects the data to be a flat list. 
In python, this is done either by using `itertools.chain.from_iterable` or python list comprehension as

`[element for sublist in superlist for element in sublist]`

In [ ]:
from itertools import chain
data = [['which', 'is', 'the', 'sense', 'of', 'life', '?'], ['the', 'answer', 'is', '42']]
list(chain.from_iterable(data))

In [ ]:
# List Comprehension
[token for sent in data for token in sent]

In [ ]:
# Or another trick
sum(data,[])

In [ ]:
from nltk.lm.preprocessing import flatten
list(flatten(data))

#### 1.1.7. Combining the Tasks
NLTK wraps both tasks:

- padded `ngram` (everygram) extraction from each sentence
- flat list creation

into a convenient utility function `padded_everygram_pipeline` that returns lazy iterators.

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline

padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [ ]:
for sent_ngrams in padded_ngrams:
    print(list(sent_ngrams))

In [ ]:
list(flat_text)

**Note:** lazy iterators (aka generators) are for one-use only. 

In [ ]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, ['winter is coming'.split()])
print(padded_ngrams)
print([list(x) for x in padded_ngrams]) # We use the generator
print([list(x) for x in padded_ngrams]) # The generator is gone 
# If you want to reuse the generator you have to recompute it
padded_ngrams, flat_text = padded_everygram_pipeline(2, ['winter is coming'.split()])
print([list(x) for x in padded_ngrams])

### 1.2. Ngram Counter
NLTK provides NgramCounter class to do the ngram counting. The class can be initialized without any argument and then updated with `ngrams` (via `update()` method); or directly initialized with `ngrams`.

`N()` method returns total number of N-grams stored.

In [ ]:
from nltk.lm import NgramCounter

counter = NgramCounter()
counter.N()  # return total number of stored ngrams

In [ ]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter.update(padded_ngrams)  # update counter with ngrams
counter.N()

In [ ]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter = NgramCounter(padded_ngrams)
counter.N()

#### 1.2.1. Accessing Ngram Counts

Ngram counts can be accessed using standard python dictionary notation. 
- Ngram order counts can be accessed using integer keys (order)
    - returns `Frequency Distribution` or `Conditional Frequency Distribution` objects. [link](https://www.nltk.org/api/nltk.probability.html)
- Unigram counts can be accessed using string keys.
- Bigram counts can be accessed using list keys & string keys (see example)

In [ ]:
# ngram order counts
print(counter[1])  # Frequency Distribution
print(counter[2])  # Conditional Frequency Distribution

In [ ]:
# print counts
print(counter.unigrams)  # unigram count for convenience

In [ ]:
# unigram counts
counter['the']

In [ ]:
# 'full' bigram counts (full bigrams)
counter[['the']]['answer']

In [ ]:
# to get a frequency distribution over all continuations, you can use list or tuple. 
# counter[['the']] or counter[('the',)]
counter[['the']]

In [ ]:
# For Conditional Frequency Distributions, only tuples are accepted.
# counter[2][['the']] with throw an error
counter[2][('the',)]

### Exercise 1

- Load Shakespeare's Hamlet from Gutenberg corpus
    - lowercase it

- Extract padded unigrams and bigrams

- Using NgramCounter
    - get total number of ngrams
    - get count of unigram `the`
    - get count of bigram `of the`
    
|                     | Count  | 
|---------------------|---------|
| Ngrams      | 84038     | 
| Unigram *the* | 993|
| Bigram *of the*     |59 |

In [ ]:
from nltk.corpus import gutenberg

# It returns a list of lists
hamlet = gutenberg.sents('shakespeare-hamlet.txt')

print(len(hamlet))

# lowercasing use .lower()
hamlet_lowercase = #lowercase the hamlet corpus

# double check
print(hamlet[0])
print(hamlet_lowercase[0])

In [ ]:
padded_ngrams, flat_text = #use the padded every gram pipeline

In [ ]:
counter = # NgramCounter(X)

In [ ]:
print(counter) # Total number of Ngrams 
print(counter[]) # the
print(counter[[]][]) # of the

## 2. Vocabulary and Basic Usage

[`Vocabulary`](https://www.nltk.org/api/nltk.lm.vocabulary.html) class of NLTK satisfies two common language modeling requirements for a vocabulary:
- When checking membership and calculating its size, filters items by comparing their counts to a cutoff value.
- Adds a special "unknown" token which unseen words are mapped to.

Tokens with counts greater than or equal to the cut-off value will be considered part of the vocabulary.

`Vocabulary(counts=None, unk_cutoff=1, unk_label='<UNK>)` create a new `Vocabulary`.

- `counts` (optional) iterable or collections. 
    - Counter instance to pre-seed the `Vocabulary`. 
    - In case it is iterable, counts are calculated.

- `unk_cutoff` (int) - Words that occur less frequently than this value are not considered part of the vocabulary.

- `unk_label` - Label for marking words not part of vocabulary.

In [ ]:
from nltk.lm import Vocabulary
from nltk.corpus import gutenberg

hamlet_words = gutenberg.words('shakespeare-hamlet.txt')

# lowercase
hamlet_words = [w.lower() for w in hamlet_words]

# initialize vocabulary with cut-off
vocab = Vocabulary(hamlet_words, unk_cutoff=2)

`Vocabulary` methods:

- `lookup()` looks up words in a vocabulary. 
    - "Unseen" words (with counts less than cutoff) are looked up as the unknown label. 
    - If given one word (a string) as an input, this method will return a string.
    - If given a sequence, it will return an tuple of the looked up words.
    
- `update()` update counts from a sequence

`Vocabulary` properties:

- `cutoff` - same as `unk_cutoff`

- `unk_label` and `counts` can also be accessed as properties
    - `vocab.unk_label`
    - `vocab.counts`

### 2.1. Counts, Vocabulary Membership, and Lookup

Tokens with frequency counts less than the `cutoff` value will be considered not part of the vocabulary, even though their entries in the count dictionary are preserved. 

This is useful for changing cut-off without recomputing counts.

In [ ]:
# let's define a function to illustrate the relation between counts, membership, and lookup
def test_word(word, vocab):
    # let's lowercase it
    if word != vocab.unk_label:
        word = word.lower() 
    # membership test
    word_membership = word in vocab
    # accessing count
    word_count = vocab[word]
    # lookup
    word_lookup = vocab.lookup(word)
    
    print("Word: '{}', Count: {}, In Vocab: {}, Mapped to: '{}'".format(
        word, word_count, word_membership, word_lookup))
    

In [ ]:
for w in ["<UNK>", "the", "1599", "Trento"]:
    test_word(w, vocab)

### 2.2. Cut-Off and Vocabulary Size
The cut-off value influences not only membership checking but also the result of getting the size of the vocabulary using the built-in `len`. Note that while the number of keys in the vocabulary's counter remains the same, the items in the vocabulary differ depending on the cut-off.

In [ ]:
# Let's define another vocabulary with cut-off 1
# Notice that we are passing counts from the original vocabulary
vocab1 = Vocabulary(vocab.counts, unk_cutoff=1)

In [ ]:
print("CutOff 2:", len(vocab))
print("CutOff 1:", len(vocab1))

In [ ]:
print("CutOff 2 Counts:", len(vocab.counts))
print("CutOff 1 Counts:", len(vocab1.counts))

#### Exercise 2
- lookup in vocabulary
    - "trento is the capital city of trentino"
- update vocabulary with "trento is the capital city of trentino"
    - do the lookup again to see the effect
- experiment with changing the cut-off value from `1` to `10`
    - do the lookup again to see the effect

In [ ]:
sentence = "trento is the capital city of trentino" # use the .split() to tokenize the sentence

# Cut-off 0 
print('Cut-off 0')
vocab = Vocabulary(hamlet_words)
# Print lookup before vocab update using vocab.lookup
# Update vocab using vocab.update
# Print lookup after vocab update using vocab.lookup

In [ ]:
print('Cut-off 1')
# Cut-off 1
# Reinitialize vocab with unk_cutoff= 1
# Print lookup before vocab update
# Update vocab
# Print lookup after vocab update

print('Cut-off 10')
# Cut-off 10
# Reinitialize vocab with unk_cutoff= 10
# Print lookup before vocab update
# Update vocab
# Print lookup after vocab update


## 3. Training Language Model

Using the data prepared as we have seen above, the `NLTK` ngram model boils down to counting ngrams, as presented above, by specifying the highest ngram size order.

In [ ]:
# Let's prepare data on hamlet
from nltk.lm.preprocessing import padded_everygram_pipeline
data = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-hamlet.txt')]
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [ ]:
# Let's train a `Maximum Likelihood Estimator (MLE)
from nltk.lm import MLE

mle_lm = MLE(2) # Where 2 is the highest N-gram size order

Initialization of the Language Model creates:
- empty vocabulary
- empty counts

These are populated once we `fit` the model with training data.

In [ ]:
# .vocab refers to the Vocabulary class that we have seen before
print(mle_lm.vocab)
# .vocab refers to the NgramCounter class that we have seen before
print(mle_lm.counts)

In [ ]:
mle_lm.fit(padded_ngrams, flat_text)

In [ ]:
print(mle_lm.vocab)
print(mle_lm.counts)

## 4. Using Ngram Language Models

A statistical [language model](https://en.wikipedia.org/wiki/Language_model) is a probability distribution over sequences of words. Given such a sequence, say of length $n$, it assigns a probability $P(w_{1},\ldots ,w_{n})$ ($P(w_{1}^{n})$, for compactness) to the whole sequence (using Chain Rule). Consequently, the unigram and bigram probabilities computed above constitute an ngram language model of our corpus.

<!-- It is more useful for Natural Language Processing to have a __probability__ of a sequence being legal, rather than a grammar's __boolean__ decision whether it is legal or not. -->

### 4.1. Computing Probability of a Sequence (Scoring)

The most common usage of a language model is to compute probability of a sequence.

#### Probability of a Sequence

Probability of a sequence is computed as a product of conditional probabilities ([Chain Rule](https://en.wikipedia.org/wiki/Chain_rule_(probability))). 

$$P(w_{1}^{n}) = P(w_1) P(w_2|w_1) P(w_3|w_1^2) ... P(w_n|w_{1}^{n-1}) = \prod_{i=1}^{n}{P(w_i|w_{1}^{i-1})}$$

The order of ngram makes a simplifying assumption that probability of a current word only depends on previous $N - 1$ elements. Thus, it truncates previous context (history) to length $N - 1$.

$$P(w_i|w_{1}^{i-1}) \approx P(w_i|w_{i-N+1}^{i-1})$$

Consequently we have:
\begin{align*}
Unigram:\;& P(w_i) \\
Bigram: \;& P(w_i|w_{i-1}) \\
Trigram: \;& P(w_i|w_{i-2},w_{i-1}) \\
\end{align*}

<!--| N-gram   | Equation                     |
:--------:|:---------------------------:|
unigram  | $P(w_i)$                     |
bigram   | $P(w_i|w_{i-1})$             |
trigram  | $P(w_i|w_{i-2},w_{i-1})$     | 
-->
The probability of the whole sequence applying an ngram model becomes:

$$P(w_{1}^{n}) = \prod_{i=1}^{n}{P(w_i|w_{i-N+1}^{i-1})}$$

#### Calculating Probability from Frequencies

Probabilities of ngrams can be computed by *normalizing* frequency counts (aka *Maximum Likelihood Estimation*): dividing the frequency of an ngram sequence by the frequency of its prefix (*relative frequency*).
\begin{align*}
Unigram:\; & P(w_i) = \frac{c(w_i)}{N}\\
Bigram:\; & P(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}\\
Ngram: \; & P(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}\\
\end{align*}

<!-- 
N-gram   | Equation                      
:--------|:------------------------------
Unigram  | $$p(w_i) = \frac{c(w_i)}{N}$$ 
Bigram   | $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$ 
Ngram    | $$p(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}$$  -->


where:
- $N$ is the total number of words in a corpus
- $c(x)$ is the count of occurrences of $x$ in a corpus (x could be unigram, bigram, etc.)

#### 4.1.1. Scoring Methods of NLTK LMs
- `score(word, context=None)` masks out of vocab (OOV) words (i.e. maps them to `<UNK>`) and computes their model score.
    - scores are **model-specific** (later on this)
    - Parameters:
        - `word (str)` - Word for which we want the score
        - `context (tuple(str))` - Context the word is in. If None, computes unigram score.
        
- `logscore(word, context=None)` - Evaluate the log score of this word in this context.

In [ ]:
# A little illustration of the methods
mle_lm.score("the")

In [ ]:

mle_lm.score("the", ["of"])


In [ ]:
import math 
mle_lm.logscore("the", ["of"]) # == math.log(mle_lm.logscore("the", ["of"]), 2) log base 2


#### Exercise 3
Implement a function to compute score of a sequence (i.e. Chain Rule)

- arguments:
    - Language Model
    - List of Tokens

- functionality
    - extracts ngrams w.r.t. LM order (`lm.order`)
    - scores each ngram w.r.t. LM (`lm.score` or `lm.logscore`)
        - note: `score` already takes care of OOV by converting to `<UNK>` 
    - computes the overall score using chain rule
        - note: the difference between `score` and `logscore`

- compute the scores of the `test_sents`
    - compute the scores of padded and unpadded sequences

In [ ]:
# Toy test set
test_sents = ["the king is dead", "the tzar is dead", 'the tragedie of hamlet is good']


In [ ]:
def chain_rule(lm, sentence, log=True, pad=True):
    highest_ngram = lm.order
    tokens = #split sentence in tokens w .split
    if pad:
        ngrams, _ = #use padded everygram pipeline with the ngram is the highest ngram
        ngrams = chain.from_iterable(ngrams) # Flat the sequence
    else:
        ngrams = # use everygrams with max len= highest ngram 
  
    
    if log:
        total_score = 0
    else:
        total_score = 1
        
    for x in ngrams:
        # x: (w_t-N, w_t-(N-1), w_t-(N-2), ... ,w_t)
        if len(x) == highest_ngram:
            if log:
                w_t = x[-1]
                # In python you can get a split of a tuple or array as array[from:to] "to" is excluded
                context = # get the slice from 0 to -2
                score = lm.logscore(w_t, context)
                total_score # Add or multiply score to total_score ?
            else:
                w_t = x[-1]
                # In python you can get a split of a tuple or array as array[from:to] "to" is excluded
                context = # get the slice from 0 to -2 
                score = lm.score(w_t, context)
                total_score # Add or multiply score to total_score ?
    
    return total_score

for sent in test_sents:
    print(sent, chain_rule(mle_lm, sent, log=True, pad=False))

#### 4.1.2. OOV in MLE
In MLE LM we did not take care of OOV. Consequently, those have `0` counts and probabilities.

In [ ]:
print(mle_lm.score("<UNK>"))
print(mle_lm.score("<UNK>", ["<UNK>"]))
print(mle_lm.score("<UNK>", ["the"]))

In [ ]:
# same as above: getting lowercaseed sentences and words 
hamlet_sents = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-hamlet.txt')]
hamlet_words = flatten(hamlet_sents)

In [ ]:
# computing vocabulary with cutoff
lex = Vocabulary(hamlet_words, unk_cutoff=2)
print(len(lex))

In [ ]:
# replacing words with counts below cutoff with '<UNK>'
hamlet_oov_sents = [list(lex.lookup(sent)) for sent in hamlet_sents]
print(hamlet_oov_sents[0])

In [ ]:
# extracting ngrams & words again
padded_ngrams_oov, flat_text_oov = padded_everygram_pipeline(2, hamlet_oov_sents)

In [ ]:
lm_oov = MLE(2)
lm_oov.fit(padded_ngrams_oov, flat_text_oov)

In [ ]:
print(lm_oov.score("<UNK>"))
print(lm_oov.score("<UNK>", ["<UNK>"]))
print(lm_oov.score("<UNK>", ["the"]))

#### 4.1.3. Data Sparsity: Missing Ngrams
However, even though we have counted ngrams on the data set with `<UNK>`, the counts still do not account for all possible ngrams. Thus, some possible sequences will have zero probability.

__We will address this later.__

In [ ]:
print(lm_oov.score("queen"))
print(lm_oov.score("<UNK>", ["queen"]))
print(lm_oov.score("queen", ["<UNK>"]))

### 4.2. Generating Sequences

Ngram Model can be used as an automaton to generate probable legal sequences using the algorithm below.

__Algorithm for Bigram LM__

- $w_{i-1} = $ `<s>`;
- *while* $w_i \neq $ `</s>`

    - stochastically get new word w.r.t. $P(w_i|w_{i-1})$

#### 4.2.1. Generation Method of NLTK LM
- `generate(num_words=1, text_seed=None, random_seed=None)` generates words from the model.
    - Parameters
        - `num_words (int)` - How many words to generate. By default 1.
        - `text_seed` - preceding context the generation should be conditioned on.
            - ngram model is restricted in how much preceding context it can take into account based on max ngram size
        - `random_seed` - A random seed. If provided, makes the random sampling part of generation reproducible.

In [ ]:
mle_lm.generate(5)

In [ ]:
# Note that you have to manually truncate the sentence when there is a </s>
mle_lm.generate(5, text_seed=["<s>", "hamlet"])

### 4.3. Language Model Evaluation

Language Models are evaluated using [Perplexity](https://en.wikipedia.org/wiki/Perplexity)

- Measures how well model fits test data
- Probability of test data
- Weighted average branching factor in predicting the next word (lower is better).
- Computed as:
\begin{align*}
 PP(W) = \sqrt[N]{\frac{1}{P(w_1,w_2,...,w_N)}} = \sqrt[N]{\frac{1}{\prod_{i=1}^{N}P(w_i|w_{i-N+1})}}
 \end{align*}

(abbreviated as PPL also)

Where $N$ is the number of words in test set.


#### 4.3.1. Evaluation Methods of NLTK LM

`NLTK` provides both **perplexity** and **cross-entropy** as evaluation methods

- `entropy(text_ngrams)` calculates cross-entropy of model for given evaluation text.
    - Parameters
        - `text_ngrams (Iterable(tuple(str)))` A sequence of ngram tuples.

- `perplexity(text_ngrams)` calculates the perplexity of the given text.
    - This is 2<sup>cross-entropy</sup> for the text, so the arguments are the same.

**Perplexity** is related to [**Cross-Entropy**](https://en.wikipedia.org/wiki/Cross_entropy) as:
\begin{align*}
PP(p) = 2^{H(p)} 
\end{align*}


#### Exercise 4
Compute entropy and perplexity of the `MLE` models  on the bigrams of the test sentences below, treating them as a test set.

- experiment with the two test sets
- experiment with OOVs (with vs without)


In [ ]:
test_sents1 = ["the king is dead", "the emperor is dead", "may the force be with you"]
test_sents2 = ["the king is dead", "welcome to you", "how are you"]

In [ ]:
# Load data
hamlet_sents = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-hamlet.txt')]
hamlet_words = flatten(hamlet_sents)
# Compute vocab 
lex = Vocabulary(hamlet_words, unk_cutoff=2)
# Handeling OOV
hamlet_oov_sents = [list(lex.lookup(sent)) for sent in hamlet_sents]
padded_ngrams_oov, flat_text_oov = padded_everygram_pipeline(2, hamlet_oov_sents)
# Train the model 
lm_oov = MLE(2)
lm_oov.fit(padded_ngrams_oov, flat_text_oov)
# Compute PPL and entropy with OOV on test 1
test_set = test_sents2

ngrams, flat_text = padded_everygram_pipeline(lm_oov.order, [lex.lookup(sent.split()) for sent in test_set])
# Compute PPL extracting ngram equals to lm_oov.order
# print(lm_oov.perplexity([# Extract ngrams == lm_oov.order))

# Generators are one-use only!
ngrams, flat_text = padded_everygram_pipeline(lm_oov.order, [lex.lookup(sent.split()) for sent in test_set])
# Compute Entropy extracting ngram equals to lm_oov.order
# print(lm_oov.entropy([# Extract ngrams == lm_oov.order ])) 


##### PPL: how it works inside

In [ ]:
import math
import numpy as np

def compute_ppl(model, data):
    highest_ngram = model.order
    scores = [] 
    for sentence in data:
        ngrams, _ = padded_everygram_pipeline(highest_ngram, [sentence.split()])
        scores.extend([-1.0 * model.logscore(w[-1], w[0:-1]) for gen in ngrams for w in gen if len(w) == highest_ngram])
    return math.pow(2.0, np.asarray(scores).mean())
compute_ppl(mle_lm, test_sents2)    


### 4.4. Handling Data Sparseness: Smoothing in NLTK

[`Smoothing(vocabulary, counter, **kwargs)`](https://www.nltk.org/api/nltk.lm.smoothing.html) Class is initialized with the following parameters: 

- `vocabulary (nltk.lm.vocab.Vocabulary)` - The Ngram vocabulary object.
- `counter (nltk.lm.counter.NgramCounter)` - The counts of the vocabulary items.


`Ngram Smoothing Interface` implements [Chen & Goodman 1995](https://aclanthology.org/P96-1041.pdf)'s idea that all smoothing algorithms have certain features in common. Consequently, each Smoothing subclass implements two methods:

- `unigram_score(word)` return unigram score
- `alpha_gamma(word, context)` returns alpha and gamma values (varies w.r.t. method)
    - `gamma` is the value added to missing ngram count
    - `alpha` is the value used to scale the lower order probabilities

The following smoothing methods are implemented:
- `WittenBell(vocabulary, counter, **kwargs)` - Witten-Bell smoothing

- `AbsoluteDiscounting(vocabulary, counter, discount=0.75, **kwargs)` - Smoothing with absolute discount
    - takes `discount=0.75` parameter (default 0.75)
    
- `KneserNey(vocabulary, counter, order, discount=0.1, **kwargs)` - Kneser-Ney Smoothing (an extension of smoothing with a discount)
    

### 4.5. NLTK Language Models

All the language models in NLTK share the same interface and share methods for
- evaluation
- generation
- scoring 

**Scoring** in a language model does the following:
- takes care of OOV words (see above)
- computes `unmasked_score`

The way `unmasked_score` is computed from counts is the difference.

Some language models use **smoothing** to account for missing ngrams, some do not (e.g. `MLE`)

Available LM Implementations

- `MLE` - providing MLE ngram model scores.

- Additive Smoothing LMs
    - `Laplace` - Laplace (add one) smoothing ($\gamma = 1$)
    - `Lidstone` - same as `Laplace`, but adds $\gamma$

- Back-Off Language Models
    - `StupidBackoff` - uses `alpha` to scale the lower order probabilities for computing missing ngram scores

- Interpolated Language Models
    - `WittenBellInterpolated` - Interpolated version of Witten-Bell smoothing
    - `KneserNeyInterpolated` - Interpolated version of Kneser-Ney smoothing.
    - `AbsoluteDiscountingInterpolated` - Interpolated version of smoothing with absolute discount

## Last Exercise: Language Model Evaluation
Write your own implementation of the Stupid backoff algorithm. Train it and compare the perplexity with the one provided by NLKT. The dataset that you have to use is the *Shakespeare Macbeth*. You have to split the dataset in training, development and test sets. The train the model on the training set, find the best ⍺ on the dev set, and test the model on the test set.

Stupid Backoff algorithm (use ⍺=0.4):
https://aclanthology.org/D07-1090.pdf 

NLTK (StupidBackoff):
https://www.nltk.org/api/nltk.lm.html

**Suggestion**: adapt the `compute_ppl` function to compute the perplexity of your model. The PPL has to be computed on the whole corpus and not at sentence level.

In [ ]:
# NLTK StupidBackoff
from nltk.lm import StupidBackoff
# Dataset
macbeth_sents = [[w.lower() for w in sent] for sent in gutenberg.sents('shakespeare-macbeth.txt')]
# Split the corpus 'shakespeare-macbeth.txt' into train, dev, test as we have seen in LAB 02